In [89]:
import pandas as pd
import numpy as np
import cvxpy as cp
import warnings

warnings.filterwarnings('error')

In [90]:
gd_r = 0.99
bc_r = 1
gate = 1.031

np.random.seed(0)


def decision(type, r2, r3, s2, s3, x1, x2, x3):  # r2,r3分别是黄金比特币增长率, s2,s3分别是黄金比特币风险, x1,x2,x3是美元黄金比特币初始值
    # 根据类型设置规划问题值
    if type == 0:  # 双买
        c1 = np.array([-r2 * gd_r, -r3 * bc_r])
        c2 = np.array([s2 * gd_r, s3 * bc_r])
        a = np.array([[1, 1]])
        b = np.array([x1])
        x = cp.Variable(2, nonneg=True)
    elif type == 1:  # 双卖
        c1 = np.array([-r2 / gd_r, -r3 / bc_r])
        c2 = np.array([s2 / gd_r, s3 / bc_r])
        a = np.array([[-1, 0], [0, -1]])
        b = np.array([gd_r * x2, bc_r * x3])
        x = cp.Variable(2, nonpos=True)
    elif type == 2:  # gold买 bitcoin卖
        c1 = np.array([-r2 * gd_r, -r3 / bc_r])
        c2 = np.array([s2 * gd_r, s3 / bc_r])
        a = np.array([[-1, 0], [0, 1], [0, -1], [1, 1]])
        b = np.array([0, 0, bc_r * x3, x1])
        x = cp.Variable(2)
    else:  # gold卖 bitcoin买
        c1 = np.array([-r2 / gd_r, -r3 * bc_r])
        c2 = np.array([s2 / gd_r, s3 * bc_r])
        a = np.array([[1, 0], [-1, 0], [0, -1], [1, 1]])
        b = np.array([0, gd_r * x2, 0, x1])
        x = cp.Variable(2)

    # 解多目标线性规划
    con = [a @ x <= b]

    obj1 = cp.Minimize(c1 @ x)
    prob1 = cp.Problem(obj1, con)
    prob1.solve(solver='SCS')
    v1 = prob1.value  # 第一个目标函数的最优值
    obj2 = cp.Minimize(c2 @ x)
    prob2 = cp.Problem(obj2, con)
    prob2.solve(solver='SCS')
    v2 = prob2.value  # 第二个目标函数的最优值
    # print('\n======结果为======\n')
    # print('两个目标函数的最优值分别为：', v1, v2)
    obj3 = cp.Minimize((c1 @ x - v1) ** 2 + (c2 @ x - v2) ** 2)
    prob3 = cp.Problem(obj3, con)
    prob3.solve(solver='ECOS_BB')  # GLPK_MI 解不了二次规划，只能用CVXOPT求解器

    # 结果
    res = x.value

    if res is None:
        return -np.inf, 0, 0, 0

    # 根据类型设置残余
    if type == 0:  # 双买
        y2 = res[0] * gd_r + x2  # 黄金当前量
        y3 = res[1] * bc_r + x3  # 比特币当前量
    elif type == 1:  # 双卖
        y2 = res[0] / gd_r + x2  # 黄金当前量
        y3 = res[1] / bc_r + x3  # 比特币当前量
    elif type == 2:  # gold买 bitcoin卖
        y2 = res[0] * gd_r + x2  # 黄金当前量
        y3 = res[1] / bc_r + x3  # 比特币当前量
    else:  # gold卖 bitcoin买
        y2 = res[0] / gd_r + x2  # 黄金当前量
        y3 = res[1] * bc_r + x3  # 比特币当前量

    y1 = x1 - res[0] - res[1]

    pred = y1 + y2 * (1 + r2) + y3 * (1 + r3)
    pred_raw = x1 + x2 * (1 + r2) + x3 * (1 + r3)

    # 不到阈值不移动
    if pred < gate * pred_raw:
        return pred_raw, x1, x2, x3
    else:
        return pred, y1, y2, y3


def choose(r2, r3, s2, s3, x1, x2, x3):
    m = -np.inf
    y1_res = x1
    y2_res = x2
    y3_res = x3  # 如果全部异常,不重新分配

    for type in range(4):  # 找最大值
        try:
            pred, y1, y2, y3 = decision(type, r2, r3, s2, s3, x1, x2, x3)
        except:
            continue

        if pred > m:
            m = pred
            y1_res = y1
            y2_res = y2
            y3_res = y3
    return m, y1_res, y2_res, y3_res


def bitcoin_only(type, r3, s3, x1, x2, x3):
    if type == 0:
        c1 = np.array([-r3 * bc_r])
        c2 = np.array([s3 * bc_r])
        a = np.array([[1], [-1]])
        b = np.array([x1, 0])
        x = cp.Variable(1)
    else:
        c1 = np.array([-r3 / bc_r])
        c2 = np.array([s3 / bc_r])
        a = np.array([[1], [-1]])
        b = np.array([0, bc_r * x3])
        x = cp.Variable(1)

    # 解多目标线性规划
    con = [a @ x <= b]

    obj1 = cp.Minimize(c1 @ x)
    prob1 = cp.Problem(obj1, con)
    prob1.solve(solver='SCS')
    v1 = prob1.value  # 第一个目标函数的最优值
    obj2 = cp.Minimize(c2 @ x)
    prob2 = cp.Problem(obj2, con)
    prob2.solve(solver='SCS')
    v2 = prob2.value  # 第二个目标函数的最优值
    # print('\n======结果为======\n')
    # print('两个目标函数的最优值分别为：', v1, v2)
    obj3 = cp.Minimize((c1 @ x - v1) ** 2 + (c2 @ x - v2) ** 2)
    prob3 = cp.Problem(obj3, con)
    prob3.solve(solver='ECOS_BB')  # GLPK_MI 解不了二次规划，只能用CVXOPT求解器

    # 结果
    res = x.value

    if res is None:
        return -np.inf, 0, 0, 0

    # 根据类型设置残余
    if type == 0:  # 买
        y3 = res[0] * bc_r + x3  # 比特币当前量
    else:  # 卖
        y3 = res[0] / bc_r + x3  # 比特币当前量

    y2 = x2
    y1 = x1 - res[0]

    pred = y1 + y2 + y3 * (1 + r3)
    pred_raw = x1 + x2 + x3 * (1 + r3)

    if pred < gate * pred_raw:
        return pred_raw, x1, x2, x3
    else:
        return pred, y1, y2, y3


def choose2(r3, s3, x1, x2, x3):
    m = -np.inf
    y1_res = x1
    y2_res = x2
    y3_res = x3  # 如果全部异常,不重新分配
    for type in range(2):  # 找最大值
        try:
            pred, y1, y2, y3 = bitcoin_only(type, r3, s3, x1, x2, x3)
        except:
            continue

        if pred > m:
            m = pred
            y1_res = y1
            y2_res = y2
            y3_res = y3
    return m, y1_res, y2_res, y3_res

In [91]:
data_g = pd.read_csv('lbma_final.csv')
data_g

,Predict,Date,USD (PM),Rate
0,NaN,2016/9/12,1324.60,1.000000
1,NaN,2016/9/13,1323.65,0.999283
2,NaN,2016/9/14,1321.75,0.998565
3,NaN,2016/9/15,1310.80,0.991716
4,NaN,2016/9/16,1308.35,0.998131
...,...,...,...,...
1260,1828.043010,2021/9/6,1821.60,0.998848
1261,1831.348043,2021/9/7,1802.15,0.989323
1262,1812.631970,2021/9/8,1786.00,0.991038
1263,1789.251128,2021/9/9,1788.25,1.001260


In [92]:
data_b = pd.read_csv('bchain_final.csv')
data_b

,Predict,Date,Value,Rate
0,NaN,2016/9/11,621.65,1.000000
1,NaN,2016/9/12,609.67,0.980729
2,NaN,2016/9/13,610.92,1.002050
3,NaN,2016/9/14,608.82,0.996563
4,NaN,2016/9/15,610.38,1.002562
...,...,...,...,...
1821,48511.52025,2021/9/6,51769.06,1.036472
1822,53319.36156,2021/9/7,52677.40,1.017546
1823,54927.64542,2021/9/8,46809.17,0.888601
1824,45943.06930,2021/9/9,46078.38,0.984388


In [93]:
# 准备装数据
res = pd.DataFrame(columns=['Date', 'C_dist', 'G_dist', 'B_dist', 'C', 'G', 'B', 'asset'])
res

,Date,C_dist,G_dist,B_dist,C,G,B,asset


In [94]:
g = 10
l = len(data_b)
# 初始拥有量
C = 1000
G = 0
B = 0

for b in range(14, l):  # 对每一个比特币交易日
    if data_b['Date'][b] == data_g['Date'][g]:  # 比特币黄金都有
        # 预计增长率
        r2 = (data_g['Predict'][g] - data_g['USD (PM)'][g - 1]) / data_g['USD (PM)'][g - 1]
        r3 = (data_b['Predict'][b] - data_b['Value'][b - 1]) / data_b['Value'][b - 1]
        # r2 = np.random.uniform(-0.05,0.05)
        # r3 = np.random.uniform(-0.05,0.05)
        # 风险
        risk_cal2 = data_g['Rate'][g - 10:g].tolist()
        risk_cal3 = data_b['Rate'][b - 14:b].tolist()
        s2 = np.std(risk_cal2)
        s3 = np.std(risk_cal3)
        # 分配
        print(data_b['Date'][b], r2, r3, s2, s3, C, G, B)
        _, C_d, G_d, B_d = choose(r2, r3, s2, s3, C, G, B)
        # 实际情况
        C = abs(round(C_d, 2))
        G = abs(round(G_d * data_g['Rate'][g], 2))
        B = abs(round(B_d * data_b['Rate'][b], 2))

        asset = C + G + B

        # res.append({'Date': data_b['Date'][b], 'C_dist': C_d, 'G_dist': G_d, 'B_dist': B_d, 'C': C, 'G': G, 'B': B,
        #             'asset': asset}, ignore_index=True)

        res.loc[len(res)] = [data_b['Date'][b], C_d, G_d, B_d, C, G, B, asset]

        g += 1
    else:  # 只有比特币
        # 预计增长率
        r3 = (data_b['Predict'][b] - data_b['Value'][b - 1]) / data_b['Value'][b - 1]
        # 风险
        risk_cal3 = data_b['Rate'][b - 14:b].tolist()
        s3 = np.std(risk_cal3)
        # 分配
        print(data_b['Date'][b], r3, s3, C, G, B)
        _, C_d, G_d, B_d = choose2(r3, s3, C, G, B)
        # 实际情况
        C = abs(round(C_d, 2))
        G = abs(round(G_d, 2))
        B = abs(round(B_d * data_b['Rate'][b], 2))

        asset = C + G + B

        res.loc[len(res)] = [data_b['Date'][b], C_d, G_d, B_d, C, G, B, asset]

2016/9/25 -0.0036403802079882632 0.008395630172790915 1000 0 0
2016/9/26 -0.010378001718148974 0.00019289011865594022 0.005219697675947488 0.008383900908302858 1000 0 0.0
2016/9/27 -0.01563112719134652 0.0024957010250759274 0.005207839507776798 0.006997360389203569 1000 0.0 0.0
2016/9/28 0.0002635847776939741 -0.0003417261865469923 0.006212593963542737 0.0077195044886382595 1000 0.0 0.0
2016/9/29 -0.0036141209829867655 0.0007782814073671335 0.006293522054098375 0.007678016148591425 1000 0.0 0.0
2016/9/30 -0.004044125635384169 0.00022950616875039575 0.005790548424542864 0.00766149912582058 1000 0.0 0.0
2016/10/1 0.0016603139204779144 0.008058719448880668 1000 0.0 0.0
2016/10/2 0.015225368725805814 0.008300696912380218 1000 0.0 0.0
2016/10/3 -0.00013686729678632034 0.013039672746255875 0.005781719329863526 0.008148583259452459 1000 0.0 0.0
2016/10/4 0.0012248960633519455 0.006879881016588999 0.0060814591961881465 0.008140035705993406 1000 0.0 0.0
2016/10/5 0.017297237590586764 0.00076211

In [95]:
res

,Date,C_dist,G_dist,B_dist,C,G,B,asset
0,2016/9/25,1000,0,0,1000,0,0.0,1000.00
1,2016/9/26,1000,0,0.0,1000,0.0,0.0,1000.00
2,2016/9/27,1000,0.0,0.0,1000,0.0,0.0,1000.00
3,2016/9/28,1000,0.0,0.0,1000,0.0,0.0,1000.00
4,2016/9/29,1000,0.0,0.0,1000,0.0,0.0,1000.00
...,...,...,...,...,...,...,...,...
1807,2021/9/6,1748.29,7590.41,0.0,1748.29,7581.67,0.0,9329.96
1808,2021/9/7,1748.29,7581.67,0.0,1748.29,7500.72,0.0,9249.01
1809,2021/9/8,1748.29,7500.72,0.0,1748.29,7433.5,0.0,9181.79
1810,2021/9/9,1748.29,7433.5,0.0,1748.29,7442.86,0.0,9191.15


In [96]:
res.to_csv('tuning/res_' + str(gd_r)+ '_' + str(bc_r) + '.csv', index=None)